In [4]:
from pyspark.sql.functions import * 
from pyspark.sql.types  import * 
from delta.tables import * 


StatementMeta(, aac42399-280c-4e32-ac63-573101231b25, 6, Finished, Available, Finished)

In [5]:
DeltaTable.createIfNotExists(spark)\
            .tableName('dimDate_gold')\
            .addColumn('OrderDate',DateType())\
            .addColumn('Day',IntegerType())\
            .addColumn('Month',IntegerType())\
            .addColumn('Year',IntegerType())\
            .addColumn('mmyyyy',StringType())\
            .addColumn('yyyymm',StringType()).execute()


sales_silver_df = spark.read.table('sales.sales_silver')

dimDate_gold_df = sales_silver_df.select('OrderDate').distinct()\
                .select('OrderDate' , \
                year('OrderDate').alias('Year'),\
                month('OrderDate').alias('Month'),\
                dayofmonth('OrderDate').alias('Day'),\
                date_format('OrderDate' , 'yyyyMM').alias('yyyymm'),\
                date_format('OrderDate' , 'MMyyyy').alias('mmyyyy')\
                )

dimDate_gold_tbl = DeltaTable.forPath(spark , 'Tables/dimdate_gold')
update_dict = {"Day": "gold_df.Day", "Month": "gold_df.Month","Year": "gold_df.Year", "mmyyyy": "gold_df.mmyyyy", "yyyymm": "gold_df.yyyymm"}

insert_dict = { "OrderDate" : "gold_df.OrderDate","Day": "gold_df.Day", "Month": "gold_df.Month","Year": "gold_df.Year", "mmyyyy": "gold_df.mmyyyy", "yyyymm": "gold_df.yyyymm"}

dimDate_gold_tbl.alias('gold_tbl')\
            .merge( dimDate_gold_df.alias('gold_df') , \
            "gold_tbl.OrderDate == gold_df.OrderDate")\
            .whenNotMatchedInsert(values =insert_dict).execute()



StatementMeta(, aac42399-280c-4e32-ac63-573101231b25, 7, Finished, Available, Finished)

In [6]:
DeltaTable.createIfNotExists(spark)\
            .tableName('dimCustomer_gold')\
            .addColumn('CustomerName',StringType())\
            .addColumn('Email',StringType())\
            .addColumn('First',StringType())\
            .addColumn('Last',StringType())\
            .addColumn('CustomerID',LongType()).execute()

dimCust_gold_df = sales_silver_df.select('CustomerName' , 'Email').distinct()\
                                .select('CustomerName' , 'Email' , \
                                split('CustomerName' , ' ').getItem(0).alias('First'),\
                                split('CustomerName' , ' ').getItem(1).alias('Last'))


dimCust_gold_tbl_tmp =spark.read.table('dimCustomer_gold')
max_customer_id = dimCust_gold_tbl_tmp.select(coalesce(max('CustomerID') , lit(0) )).alias('max_customer_id').first()[0]
dfdimCustomer_gold = dimCust_gold_df.join(dimCust_gold_tbl_tmp ,\
                                (dimCust_gold_df.CustomerName == dimCust_gold_tbl_tmp.CustomerName) & \
                                (dimCust_gold_df.Email == dimCust_gold_tbl_tmp.Email) ,'left_anti')

# display(dfdimCustomer_gold)

dfdimCustomer_gold= dfdimCustomer_gold.withColumn('CustomerId' , monotonically_increasing_id()+max_customer_id+1)

display(dfdimCustomer_gold)




dimCust_gold_tbl = DeltaTable.forPath(spark , 'Tables/dimcustomer_gold')
update_dict = {"Email": "gold_df.Email", "First": "gold_df.First","Last": "gold_df.Last"}

insert_dict = { "CustomerName" : "gold_df.CustomerName","Email": "gold_df.Email", "First": "gold_df.First","Last": "gold_df.Last", "CustomerID": "gold_df.CustomerID"}

dimCust_gold_tbl.alias('gold_tbl')\
            .merge( dfdimCustomer_gold.alias('gold_df') , \
            "gold_tbl.CustomerName == gold_df.CustomerName and gold_tbl.Email == gold_df.Email")\
            .whenNotMatchedInsert(values =insert_dict).execute()
            
            


StatementMeta(, aac42399-280c-4e32-ac63-573101231b25, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, aa5f103e-9c7a-4c2c-b8b2-85665b53fadb)

In [7]:
DeltaTable.createIfNotExists(spark)\
            .tableName('sales.dimProduct_gold')\
            .addColumn('ItemName',StringType())\
            .addColumn('ItemId',LongType())\
            .addColumn('ItemInfo',StringType()).execute()


dimProd_gold_df = sales_silver_df.select('Item').distinct()\
                                .select( \
                                split('Item' , ',').getItem(0).alias('ItemName'),\
                                when( ( (split('Item' , ',').getItem(1).isNull()) | (split('Item' , ' ').getItem(1)=='') ) , 'NULL' )\
                                .otherwise(split('Item' , ',').getItem(1) ).alias('ItemInfo'))




dimProd_gold_tbl_tmp =spark.read.table('dimProduct_gold')
max_Item_Id = dimProd_gold_tbl_tmp.select(coalesce(max('ItemId') , lit(0) )).alias('max_Item_Id').first()[0]
dfdimProd_gold = dimProd_gold_df.join(dimProd_gold_tbl_tmp ,\
                                (dimProd_gold_df.ItemName == dimProd_gold_tbl_tmp.ItemName) & \
                                (dimProd_gold_df.ItemInfo == dimProd_gold_tbl_tmp.ItemInfo) ,'left_anti')

# display(dfdimCustomer_gold)

dfdimProd_gold= dfdimProd_gold.withColumn('ItemId' , monotonically_increasing_id()+max_Item_Id+1)

display(dfdimProd_gold)



dimProd_gold_tbl = DeltaTable.forPath(spark , 'Tables/dimproduct_gold')
update_dict = {}

insert_dict = { "ItemName" : "gold_df.ItemName","ItemInfo": "gold_df.ItemInfo"}

dimProd_gold_tbl.alias('gold_tbl')\
            .merge( dfdimProd_gold.alias('gold_df') , \
            "gold_tbl.ItemName == gold_df.ItemName and gold_tbl.ItemInfo == gold_df.ItemInfo")\
            .whenNotMatchedInsert(values =insert_dict).execute()


StatementMeta(, aac42399-280c-4e32-ac63-573101231b25, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2eebb4b2-b5c8-4fb2-8c2b-99ee9e275076)

In [25]:
DeltaTable.createIfNotExists(spark)\
            .tableName('sales.factSales_gold')\
            .addColumn('CustomerID',LongType())\
            .addColumn('ItemId',LongType())\
            .addColumn('OrderDate',DateType())\
            .addColumn('Quantity',IntegerType())\
            .addColumn('UnitPrice',LongType())\
            .addColumn('Tax',IntegerType()).execute()


StatementMeta(, aac42399-280c-4e32-ac63-573101231b25, 27, Finished, Available, Finished)

In [9]:
df_dim_cust_tmp = spark.read.table('sales.dimcustomer_gold')
df_dim_prod_tmp = spark.read.table('sales.dimproduct_gold')


sales_silver_df = sales_silver_df.withColumn(\
                                'ItemName',split('Item' , ',').getItem(0))\
                                .withColumn('ItemInfo' , \
                                when( ( (split('Item' , ',').getItem(1).isNull()) | (split('Item' , ' ').getItem(1)=='') ) , 'NULL' )\
                                .otherwise(split('Item' , ',').getItem(1) ) )


display(sales_silver_df.head(10))



StatementMeta(, aac42399-280c-4e32-ac63-573101231b25, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 29cabb7e-ea33-4204-8bee-001d3997c2ce)

In [22]:
#########################################

df_dim_cust_tmp = spark.read.table('sales.dimcustomer_gold')
df_dim_prod_tmp = spark.read.table('sales.dimproduct_gold')


sales_silver_df = sales_silver_df.withColumn(\
                                'ItemName',split('Item' , ',').getItem(0))\
                                .withColumn('ItemInfo' , \
                                when( ( (split('Item' , ',').getItem(1).isNull()) | (split('Item' , ' ').getItem(1)=='') ) , 'NULL' )\
                                .otherwise(split('Item' , ',').getItem(1) ) )


# display(sales_silver_df.head(10))

factSale_gold_df = sales_silver_df.alias('silver').join(df_dim_cust_tmp.alias('cust') , ( sales_silver_df.CustomerName == df_dim_cust_tmp.CustomerName) & (sales_silver_df.Email == df_dim_cust_tmp.Email) , "left")\
                    .select('cust.CustomerName')




factSale_gold_df = sales_silver_df.alias('silver').join(df_dim_cust_tmp.alias('cust') , ( sales_silver_df.CustomerName == df_dim_cust_tmp.CustomerName) & (sales_silver_df.Email == df_dim_cust_tmp.Email) , "left")\
                                  .join(df_dim_prod_tmp.alias('prod') , (sales_silver_df.ItemName == df_dim_prod_tmp.ItemName) & (sales_silver_df.ItemInfo == df_dim_prod_tmp.ItemInfo) , "left")\
                                  .select(col('cust.CustomerID'),\
                                  col('prod.ItemId'),\
                                  col('silver.OrderDate'),\
                                  col('silver.Tax'),\
                                  col('silver.UnitPrice'),\
                                  col('silver.Quantity'))

                                  

display(factSale_gold_df.head(10))
#########################################

StatementMeta(, aac42399-280c-4e32-ac63-573101231b25, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1a6b5f99-9f76-45ea-81f7-98fd977fb239)

In [26]:
factSales_gold_tbl = DeltaTable.forPath(spark , 'Tables/factsales_gold')
update_dict = {}

insert_dict = { "CustomerID" : "gold_df.CustomerID","ItemId": "gold_df.ItemId" , "OrderDate" : "gold_df.OrderDate","Quantity":"gold_df.Quantity" , \
                "UnitPrice":"gold_df.UnitPrice" , "Tax" : "gold_df.Tax" }

factSales_gold_tbl.alias('gold_tbl')\
            .merge( factSale_gold_df.alias('gold_df') , \
            "gold_tbl.CustomerID == gold_df.CustomerID and gold_tbl.ItemId == gold_df.ItemId\
            and gold_tbl.OrderDate == gold_df.OrderDate") \
            .whenNotMatchedInsert(values =insert_dict).execute()

StatementMeta(, aac42399-280c-4e32-ac63-573101231b25, 28, Finished, Available, Finished)